In [ ]:
import matplotlib

In [ ]:
import sys
#Rename according to your virtual - env name
sys.path.append('/usr/local/anaconda3/envs/mob-reg/lib/python3.7/site-packages')

In [ ]:
import tensorflow as tf

# Extracting VGG keypoints trial

In [ ]:
from __future__ import print_function
import Registration
import matplotlib.pyplot as plt
from utils.utils import *
import cv2

# designate image path here
IX_path = '/Users/karthik/Downloads/a.jpg'
IY_path = '/Users/karthik/Downloads/b.jpg'

IX = cv2.imread(IX_path)
IY = cv2.imread(IY_path)
print(IX)
#initialize
reg = Registration.CNN()
#register
X, Y, Z = reg.register(IX, IY)
#generate regsitered image using TPS
registered = tps_warp(Y, Z, IY, IX.shape)
cb = checkboard(IX, registered, 11)

plt.subplot(141)
plt.title('reference')
plt.imshow(cv2.cvtColor(IX, cv2.COLOR_BGR2RGB))

plt.subplot(142)
plt.title('other')
plt.imshow(cv2.cvtColor(IY, cv2.COLOR_BGR2RGB))

plt.subplot(143)
plt.title('registered')
plt.imshow(cv2.cvtColor(registered, cv2.COLOR_BGR2RGB))
plt.subplot(144)
plt.title('checkboard')
plt.imshow(cv2.cvtColor(cb, cv2.COLOR_BGR2RGB))

plt.show()

In [ ]:
from __future__ import print_function
import MobileNetRegistration
import matplotlib.pyplot as plt
from utils.utils import *
import cv2

# designate image path here
IX_path = '/Users/karthik/Downloads/a.jpg'
IY_path = '/Users/karthik/Downloads/b.jpg'

IX = cv2.imread(IX_path)
IY = cv2.imread(IY_path)
print(IX)
#initialize
reg = MobileNetRegistration.CNN()
#register
hello= reg.register(IX, IY)


In [ ]:
import MobileNetRegistration

In [ ]:
A.shape

In [ ]:
F.shape

In [ ]:
reg.register(IX, IY)[2].shape

In [ ]:
IX.shape

In [ ]:
np.expand_dims(IX, axis=0).shape

In [ ]:
np.concatenate((np.expand_dims(IY, axis=0),np.expand_dims(IY, axis=0)),axis=0).shape

In [ ]:
reg.register(IX, IY)

In [ ]:
F.shape

In [ ]:
F[48]

In [ ]:
import cv2 as cv

In [ ]:
akaze = cv.AKAZE_create()
# Find the keypoints and descriptors with SIFT
kp1, des1 = akaze.detectAndCompute(IX, None)
kp2, des2 = akaze.detectAndCompute(IY, None)

In [ ]:
des1.shape

In [ ]:
hello[0].shape

In [ ]:
hello[5].shape

In [ ]:
bf = cv.BFMatcher()
matches = bf.knnMatch(hello[4], hello[5], k=2)

In [ ]:
good_matches = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good_matches.append([m])

In [ ]:
img3 = cv.drawMatchesKnn(IX,kp1,IY,kp2,good_matches,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv.imwrite('matches.jpg', img3)

In [ ]:
plt.imshow(img3)

# Mobilenet Implementation with keras

In [ ]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

In [ ]:
tf.__version__

## Preprocessing

In [ ]:
mobile = keras.applications.mobilenet.MobileNet()
def prepare_image(file):
    img_path = '/Users/karthik/Downloads/'
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
preprocessed_image = prepare_image('a.jpg')
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

In [ ]:
!ls

In [ ]:
img_path = '/Users/karthik/Downloads/'
img = image.load_img(img_path + 'a.jpg', target_size=(224, 224))
img_array = image.img_to_array(img)
img_array_expanded_dims = np.expand_dims(img_array, axis=0)

In [ ]:
img_array.shape

In [ ]:
img_array_expanded_dims.shape

In [ ]:
img_array.shape

In [ ]:
mobile.summary()

In [ ]:
get_3rd_layer_output = K.function([mobile.layers[0].input],
                                  [mobile.layers[3].output])

In [ ]:
layer_output = (get_3rd_layer_output(preprocessed_image))

In [ ]:
(layer_output[0]).shape